In [1]:
from firebase_admin import credentials, firestore, initialize_app, get_app
from algoliasearch.search_client import SearchClient

In [2]:
def initialize_firestore():
    try:
        # Kiểm tra xem ứng dụng Firebase đã được khởi tạo chưa
        get_app()
    except Exception as e:
        # Nếu chưa, khởi tạo nó
        cred = credentials.Certificate("key.json")
        initialize_app(cred)
        print("Initialize: Firebase app has been initialized.")
    return firestore.client()

In [3]:
db = initialize_firestore()

Initialize: Firebase app has been initialized.


#### Init Algolia Search

In [4]:
client = SearchClient.create("8SYE9PPQXH", "320f3dea5088a0f273a764527b790978")
user_index = client.init_index("users")
user_index.set_settings({
    "searchableAttributes": ["display_name", "email"]
})

In [15]:
# Lưu user vào algolia
user_ref = db.collection("users")
users = user_ref.stream()
for user in users:
    user_data = user.to_dict()
    user_index.save_object({
        "objectID": user.id,
        "display_name": user_data["display_name"],
        "email": user_data["email"],
        "picture": user_data["picture"],
    })

In [5]:
# Your object
user_object = {
    'objectID': '110426175374584859787',  # Unique identifier
    'email': 'taiphan2403a@gmail.com',
    'display_name': 'Nhokzuize2k3',
    'picture': 'https://lh3.googleusercontent.com/a'
}

# Insert the object into Algolia index
user_index.save_object(user_object)

#### Thử truy vấn với Algolia Search

In [7]:
query = "manh"
'email:*{}* OR display_name:*{}*'.format(query, query)

'email:*manh* OR display_name:*manh*'

In [9]:
query = "taiphan"
search_result = user_index.search(query, {
    'filters': 'email:*{}* OR display_name:*{}*'.format(query, query)
})

hits = search_result['hits']
results = [
        {
            "email": hit["email"],
            "display_name": hit["display_name"],
            "picture": hit["picture"],
        }
        for hit in hits
    ]
results
# print(hits)

[]